# Summary:
- No SMOTE
- Vector size 16x16
- Below networks are tested with below results:


    - SVM 
                         precision    recall  f1-score   support

        0.0               0.86      0.83      0.85      2645
        1.0               0.80      0.84      0.82      2133

        avg / total       0.84      0.84      0.84      4778

        Confusion Matrix
            2203  442
             344 1789

    - DNN + SVM
                         precision    recall  f1-score   support

        0.0               0.88      0.83      0.85      2645
        1.0               0.80      0.87      0.83      2133

        avg / total       0.85      0.84      0.84      4778

        Confusion Matrix
            2184  461
             287 1846
               
    - Deeper DNN with dropout + SVM
                         precision    recall  f1-score   support

        0.0               0.89      0.82      0.85      2645
        1.0               0.80      0.88      0.84      2133

        avg / total       0.85      0.85      0.85      4778

        Confusion Matrix
            2169  476
             261 1872
    
    - Inception Batch Normalization + SVM
                         precision    recall  f1-score   support

        0.0               0.86      0.83      0.85      2645
        1.0               0.80      0.83      0.82      2133

        avg / total       0.83      0.83      0.83      4778

        Confusion Matrix
            2206  439
             361 1772

In [1]:
import glob
import multiprocessing
import time
import csv
import pickle
import os
import pandas as pd
import shutil
import numpy as np
import warnings
import tqdm
import logging
import mxnet as mx
from bs4 import BeautifulSoup
from urllib.request import urlopen
from os.path import basename
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/mxnet/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


Populating the interactive namespace from numpy and matplotlib


In [2]:
# 2d to 4d conversion
def to4d(img):
    return img.reshape(img.shape[0], 1, 16, 16).astype(np.float32)

In [3]:
def modelMetric(mod, info, nd=True):
    if nd == True: 
        prob = mod.predict(val_iters).asnumpy()
    else:
        prob = mod.predict(val_iters)
    logging.info(info)
    # Check the accuracy from prediction
    val_iters.reset()

    y_batch = []
    for dbatch in val_iters:
        label = dbatch.label[0].asnumpy()
        pad = val_iters.getpad()
        real_size = label.shape[0] - pad
        y_batch.append(label[0:real_size])
    y = np.concatenate(y_batch)

    # get prediction label 
    py = np.argmax(prob, axis=1)
    acc1 = sum(py == y) / len(y)
    logging.info('Test accuracy = %f', acc1)
    cfn_matrix=confusion_matrix(y,py)
    print(classification_report (y,py))
    print(cfn_matrix)

# 1. Data Preparation

In [4]:
p_file = 'RCT_Vectors'
with open(p_file, 'rb') as fin:
    vectors = pickle.load(fin)

In [5]:
p_file = 'RCT_labels'
with open(p_file, 'rb') as fin:
    data_Y = pickle.load(fin)

In [6]:
d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(vectors, data_Y, test_size=0.25)

In [7]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [8]:
# double check whether oversampling works
X_train_array.shape, Y_train_array.shape, X_test_array.shape, Y_test_array.shape, len(vectors)

((14333, 256), (14333,), (4778, 256), (4778,), 19111)

In [9]:
# creating 4d training/testing dataset
batch_size = 64
train_iters = mx.io.NDArrayIter(to4d(X_train_array), Y_train_array, batch_size, shuffle=True,label_name='svm_label')
val_iters = mx.io.NDArrayIter(to4d(X_test_array), Y_test_array, batch_size,label_name='svm_label')

# 2. Train models
## 2.1 SVM

In [10]:
# Single SVM
data = mx.symbol.Variable('data')
svm  = mx.symbol.FullyConnected(data = data, name='fc', num_hidden=2)

# Here we add the ultimate layer based on L2-SVM objective
ssvm = mx.symbol.SVMOutput(data=svm, name='svm')
# draw network chart
shape = {"data" : (batch_size, 1, 16, 16)}
mx.viz.plot_network(symbol=ssvm, shape=shape)
network_chart=mx.viz.plot_network(symbol=ssvm, shape=shape)
network_chart.format = 'png'
network_chart.render('SVM')

'SVM.png'

<img src='SVM.png'>

In [11]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

mod = mx.model.FeedForward(
    ctx = mx.gpu(0),     # use GPU 0 for training, others are same as before
    symbol = ssvm,       # network structure
    num_epoch = 300,     # number of data passes for training 
    learning_rate = 0.00001 # learning rate of SGD 
)
mod.fit(
    X=train_iters,       # training data
    eval_data=val_iters, # validation data
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size) # output progress for each 200 data batches
)


INFO:root:Start training with [gpu(0)]
INFO:root:Epoch[0] Batch [64]	Speed: 53970.94 samples/sec	accuracy=0.576904
INFO:root:Epoch[0] Batch [128]	Speed: 62286.75 samples/sec	accuracy=0.585449
INFO:root:Epoch[0] Batch [192]	Speed: 62017.61 samples/sec	accuracy=0.602783
INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.258
INFO:root:Epoch[0] Validation-accuracy=0.597708
INFO:root:Epoch[1] Batch [64]	Speed: 67512.09 samples/sec	accuracy=0.622314
INFO:root:Epoch[1] Batch [128]	Speed: 63715.04 samples/sec	accuracy=0.617676
INFO:root:Epoch[1] Batch [192]	Speed: 59892.73 samples/sec	accuracy=0.646729
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.234
INFO:root:Epoch[1] Validation-accuracy=0.637500
INFO:root:Epoch[2] Batch [64]	Speed: 62198.80 samples/sec	accuracy=0.659180
INFO:root:Epoch[2] Batch [128]	Speed: 61716.39 samples/sec	accuracy=0.654541
INFO:root:Epoch[2] Batch [192]	Speed: 56364.77 samples/sec	accuracy=0.674072
INFO:root:Epoch[2] 

INFO:root:Epoch[22] Time cost=0.231
INFO:root:Epoch[22] Validation-accuracy=0.786667
INFO:root:Epoch[23] Batch [64]	Speed: 68210.90 samples/sec	accuracy=0.793457
INFO:root:Epoch[23] Batch [128]	Speed: 70888.09 samples/sec	accuracy=0.805664
INFO:root:Epoch[23] Batch [192]	Speed: 62236.21 samples/sec	accuracy=0.798340
INFO:root:Epoch[23] Resetting Data Iterator
INFO:root:Epoch[23] Time cost=0.221
INFO:root:Epoch[23] Validation-accuracy=0.789167
INFO:root:Epoch[24] Batch [64]	Speed: 69233.47 samples/sec	accuracy=0.795166
INFO:root:Epoch[24] Batch [128]	Speed: 67309.21 samples/sec	accuracy=0.806396
INFO:root:Epoch[24] Batch [192]	Speed: 65124.60 samples/sec	accuracy=0.800293
INFO:root:Epoch[24] Resetting Data Iterator
INFO:root:Epoch[24] Time cost=0.219
INFO:root:Epoch[24] Validation-accuracy=0.789792
INFO:root:Epoch[25] Batch [64]	Speed: 64761.51 samples/sec	accuracy=0.797119
INFO:root:Epoch[25] Batch [128]	Speed: 63742.23 samples/sec	accuracy=0.808594
INFO:root:Epoch[25] Batch [192]	Spee

INFO:root:Epoch[45] Batch [192]	Speed: 67846.43 samples/sec	accuracy=0.825439
INFO:root:Epoch[45] Resetting Data Iterator
INFO:root:Epoch[45] Time cost=0.231
INFO:root:Epoch[45] Validation-accuracy=0.816667
INFO:root:Epoch[46] Batch [64]	Speed: 62860.39 samples/sec	accuracy=0.827393
INFO:root:Epoch[46] Batch [128]	Speed: 62859.01 samples/sec	accuracy=0.829346
INFO:root:Epoch[46] Batch [192]	Speed: 66277.80 samples/sec	accuracy=0.825928
INFO:root:Epoch[46] Resetting Data Iterator
INFO:root:Epoch[46] Time cost=0.229
INFO:root:Epoch[46] Validation-accuracy=0.816667
INFO:root:Epoch[47] Batch [64]	Speed: 64448.11 samples/sec	accuracy=0.828125
INFO:root:Epoch[47] Batch [128]	Speed: 62602.46 samples/sec	accuracy=0.830078
INFO:root:Epoch[47] Batch [192]	Speed: 65964.28 samples/sec	accuracy=0.826172
INFO:root:Epoch[47] Resetting Data Iterator
INFO:root:Epoch[47] Time cost=0.229
INFO:root:Epoch[47] Validation-accuracy=0.817708
INFO:root:Epoch[48] Batch [64]	Speed: 66382.80 samples/sec	accuracy=0

INFO:root:Epoch[68] Batch [128]	Speed: 62276.36 samples/sec	accuracy=0.841553
INFO:root:Epoch[68] Batch [192]	Speed: 61644.64 samples/sec	accuracy=0.837158
INFO:root:Epoch[68] Resetting Data Iterator
INFO:root:Epoch[68] Time cost=0.232
INFO:root:Epoch[68] Validation-accuracy=0.828542
INFO:root:Epoch[69] Batch [64]	Speed: 60931.46 samples/sec	accuracy=0.837158
INFO:root:Epoch[69] Batch [128]	Speed: 64830.67 samples/sec	accuracy=0.841797
INFO:root:Epoch[69] Batch [192]	Speed: 68160.02 samples/sec	accuracy=0.837158
INFO:root:Epoch[69] Resetting Data Iterator
INFO:root:Epoch[69] Time cost=0.226
INFO:root:Epoch[69] Validation-accuracy=0.828542
INFO:root:Epoch[70] Batch [64]	Speed: 65517.26 samples/sec	accuracy=0.838135
INFO:root:Epoch[70] Batch [128]	Speed: 65568.27 samples/sec	accuracy=0.841797
INFO:root:Epoch[70] Batch [192]	Speed: 65038.80 samples/sec	accuracy=0.837158
INFO:root:Epoch[70] Resetting Data Iterator
INFO:root:Epoch[70] Time cost=0.228
INFO:root:Epoch[70] Validation-accuracy=

INFO:root:Epoch[90] Validation-accuracy=0.834583
INFO:root:Epoch[91] Batch [64]	Speed: 63942.23 samples/sec	accuracy=0.841309
INFO:root:Epoch[91] Batch [128]	Speed: 59768.12 samples/sec	accuracy=0.845703
INFO:root:Epoch[91] Batch [192]	Speed: 61867.30 samples/sec	accuracy=0.841553
INFO:root:Epoch[91] Resetting Data Iterator
INFO:root:Epoch[91] Time cost=0.239
INFO:root:Epoch[91] Validation-accuracy=0.834583
INFO:root:Epoch[92] Batch [64]	Speed: 63591.70 samples/sec	accuracy=0.841553
INFO:root:Epoch[92] Batch [128]	Speed: 60269.67 samples/sec	accuracy=0.845459
INFO:root:Epoch[92] Batch [192]	Speed: 64996.97 samples/sec	accuracy=0.841309
INFO:root:Epoch[92] Resetting Data Iterator
INFO:root:Epoch[92] Time cost=0.234
INFO:root:Epoch[92] Validation-accuracy=0.834792
INFO:root:Epoch[93] Batch [64]	Speed: 65611.34 samples/sec	accuracy=0.841553
INFO:root:Epoch[93] Batch [128]	Speed: 63558.29 samples/sec	accuracy=0.845215
INFO:root:Epoch[93] Batch [192]	Speed: 63638.57 samples/sec	accuracy=0.8

...

INFO:root:Epoch[203] Batch [64]	Speed: 65203.69 samples/sec	accuracy=0.849609
INFO:root:Epoch[203] Batch [128]	Speed: 61774.31 samples/sec	accuracy=0.852295
INFO:root:Epoch[203] Batch [192]	Speed: 61497.24 samples/sec	accuracy=0.851074
INFO:root:Epoch[203] Resetting Data Iterator
INFO:root:Epoch[203] Time cost=0.235
INFO:root:Epoch[203] Validation-accuracy=0.838958
INFO:root:Epoch[204] Batch [64]	Speed: 64810.62 samples/sec	accuracy=0.849609
INFO:root:Epoch[204] Batch [128]	Speed: 71483.60 samples/sec	accuracy=0.852295
INFO:root:Epoch[204] Batch [192]	Speed: 67914.82 samples/sec	accuracy=0.851074
INFO:root:Epoch[204] Resetting Data Iterator
INFO:root:Epoch[204] Time cost=0.216
INFO:root:Epoch[204] Validation-accuracy=0.838958
INFO:root:Epoch[205] Batch [64]	Speed: 75384.69 samples/sec	accuracy=0.849854
INFO:root:Epoch[205] Batch [128]	Speed: 71814.69 samples/sec	accuracy=0.852295
INFO:root:Epoch[205] Batch [192]	Speed: 72336.60 samples/sec	accuracy=0.851074
INFO:root:Epoch[205] Resetti

INFO:root:Epoch[225] Batch [192]	Speed: 65842.93 samples/sec	accuracy=0.850830
INFO:root:Epoch[225] Resetting Data Iterator
INFO:root:Epoch[225] Time cost=0.228
INFO:root:Epoch[225] Validation-accuracy=0.839375
INFO:root:Epoch[226] Batch [64]	Speed: 65792.75 samples/sec	accuracy=0.850098
INFO:root:Epoch[226] Batch [128]	Speed: 73861.10 samples/sec	accuracy=0.853516
INFO:root:Epoch[226] Batch [192]	Speed: 65657.73 samples/sec	accuracy=0.850830
INFO:root:Epoch[226] Resetting Data Iterator
INFO:root:Epoch[226] Time cost=0.221
INFO:root:Epoch[226] Validation-accuracy=0.839375
INFO:root:Epoch[227] Batch [64]	Speed: 70901.54 samples/sec	accuracy=0.850098
INFO:root:Epoch[227] Batch [128]	Speed: 61882.46 samples/sec	accuracy=0.853516
INFO:root:Epoch[227] Batch [192]	Speed: 66245.86 samples/sec	accuracy=0.850830
INFO:root:Epoch[227] Resetting Data Iterator
INFO:root:Epoch[227] Time cost=0.221
INFO:root:Epoch[227] Validation-accuracy=0.839375
INFO:root:Epoch[228] Batch [64]	Speed: 77239.97 sampl

INFO:root:Epoch[247] Time cost=0.231
INFO:root:Epoch[247] Validation-accuracy=0.838125
INFO:root:Epoch[248] Batch [64]	Speed: 62649.95 samples/sec	accuracy=0.849365
INFO:root:Epoch[248] Batch [128]	Speed: 65759.01 samples/sec	accuracy=0.853516
INFO:root:Epoch[248] Batch [192]	Speed: 67940.87 samples/sec	accuracy=0.850830
INFO:root:Epoch[248] Resetting Data Iterator
INFO:root:Epoch[248] Time cost=0.226
INFO:root:Epoch[248] Validation-accuracy=0.838125
INFO:root:Epoch[249] Batch [64]	Speed: 70686.54 samples/sec	accuracy=0.848877
INFO:root:Epoch[249] Batch [128]	Speed: 64161.21 samples/sec	accuracy=0.853516
INFO:root:Epoch[249] Batch [192]	Speed: 66106.42 samples/sec	accuracy=0.850586
INFO:root:Epoch[249] Resetting Data Iterator
INFO:root:Epoch[249] Time cost=0.221
INFO:root:Epoch[249] Validation-accuracy=0.837708
INFO:root:Epoch[250] Batch [64]	Speed: 66690.74 samples/sec	accuracy=0.848877
INFO:root:Epoch[250] Batch [128]	Speed: 64652.08 samples/sec	accuracy=0.853760
INFO:root:Epoch[250]

INFO:root:Epoch[270] Batch [128]	Speed: 66917.26 samples/sec	accuracy=0.853027
INFO:root:Epoch[270] Batch [192]	Speed: 66665.12 samples/sec	accuracy=0.851807
INFO:root:Epoch[270] Resetting Data Iterator
INFO:root:Epoch[270] Time cost=0.215
INFO:root:Epoch[270] Validation-accuracy=0.836250
INFO:root:Epoch[271] Batch [64]	Speed: 64269.94 samples/sec	accuracy=0.849365
INFO:root:Epoch[271] Batch [128]	Speed: 65587.04 samples/sec	accuracy=0.853027
INFO:root:Epoch[271] Batch [192]	Speed: 64770.55 samples/sec	accuracy=0.851807
INFO:root:Epoch[271] Resetting Data Iterator
INFO:root:Epoch[271] Time cost=0.226
INFO:root:Epoch[271] Validation-accuracy=0.836042
INFO:root:Epoch[272] Batch [64]	Speed: 68636.02 samples/sec	accuracy=0.849365
INFO:root:Epoch[272] Batch [128]	Speed: 70221.95 samples/sec	accuracy=0.853271
INFO:root:Epoch[272] Batch [192]	Speed: 63334.29 samples/sec	accuracy=0.851562
INFO:root:Epoch[272] Resetting Data Iterator
INFO:root:Epoch[272] Time cost=0.219
INFO:root:Epoch[272] Val

INFO:root:Epoch[292] Resetting Data Iterator
INFO:root:Epoch[292] Time cost=0.203
INFO:root:Epoch[292] Validation-accuracy=0.835833
INFO:root:Epoch[293] Batch [64]	Speed: 73145.67 samples/sec	accuracy=0.849365
INFO:root:Epoch[293] Batch [128]	Speed: 70143.10 samples/sec	accuracy=0.853027
INFO:root:Epoch[293] Batch [192]	Speed: 73099.61 samples/sec	accuracy=0.851807
INFO:root:Epoch[293] Resetting Data Iterator
INFO:root:Epoch[293] Time cost=0.206
INFO:root:Epoch[293] Validation-accuracy=0.836042
INFO:root:Epoch[294] Batch [64]	Speed: 65563.51 samples/sec	accuracy=0.849365
INFO:root:Epoch[294] Batch [128]	Speed: 67064.59 samples/sec	accuracy=0.852783
INFO:root:Epoch[294] Batch [192]	Speed: 68643.69 samples/sec	accuracy=0.851807
INFO:root:Epoch[294] Resetting Data Iterator
INFO:root:Epoch[294] Time cost=0.219
INFO:root:Epoch[294] Validation-accuracy=0.836042
INFO:root:Epoch[295] Batch [64]	Speed: 69483.24 samples/sec	accuracy=0.849365
INFO:root:Epoch[295] Batch [128]	Speed: 68896.96 sampl

In [12]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'SVM'
epoch = 300
mod.save(prefix, epoch)

INFO:root:Saved checkpoint to "SVM-0300.params"


In [13]:
modelMetric(mod, 'Finish predict of one Layer SVM Network...', False)

INFO:root:Finish predict of one Layer SVM Network...
INFO:root:Test accuracy = 0.835496


             precision    recall  f1-score   support

        0.0       0.86      0.83      0.85      2645
        1.0       0.80      0.84      0.82      2133

avg / total       0.84      0.84      0.84      4778

[[2203  442]
 [ 344 1789]]


## 2.2 DNN + SVM

In [14]:
# DNN + SVM 
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=512)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 512)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")

fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=2)

dnnsvm = mx.symbol.SVMOutput(data=fc3, name='svm')

# draw network chart
mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart=mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart.format = 'png'
network_chart.render('DNN + SVM')

'DNN + SVM.png'

<img src='DNN + SVM.png'>

In [15]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = dnnsvm,        
    label_names = ['svm_label']
)

lr_sch = mx.lr_scheduler.FactorScheduler(step=1800, factor=0.9)

mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size), 
    num_epoch = 300,       # Train for 100 epochs
    optimizer = 'Adam',
    optimizer_params = {
        'learning_rate': 9e-7, # Learning rate
        'beta1': 0.9,       # Beta 1
        'beta2': 0.999,         # Beta 2
        'lr_scheduler': lr_sch #learning rate decrease schedule
    },
)
print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 44655.98 samples/sec	accuracy=0.504087
INFO:root:Epoch[0] Batch [128]	Speed: 42129.10 samples/sec	accuracy=0.588135
INFO:root:Epoch[0] Batch [192]	Speed: 42407.07 samples/sec	accuracy=0.681396
INFO:root:Epoch[0] Train-accuracy=0.677923
INFO:root:Epoch[0] Time cost=0.342
INFO:root:Epoch[0] Validation-accuracy=0.725208
INFO:root:Epoch[1] Batch [64]	Speed: 44595.12 samples/sec	accuracy=0.755048
INFO:root:Epoch[1] Batch [128]	Speed: 43353.62 samples/sec	accuracy=0.769287
INFO:root:Epoch[1] Batch [192]	Speed: 43439.56 samples/sec	accuracy=0.802979
INFO:root:Epoch[1] Train-accuracy=0.761593
INFO:root:Epoch[1] Time cost=0.330
INFO:root:Epoch[1] Validation-accuracy=0.797708
INFO:root:Epoch[2] Batch [64]	Speed: 47622.38 samples/sec	accuracy=0.807212
INFO:root:Epoch[2] Batch [128]	Speed: 47316.64 samples/sec	accuracy=0.810791
INFO:root:Epoch[2] Batch [192]	Speed: 44721.31 samples/sec	accuracy=0.817871
INFO:root:Epoch[2] Train-accuracy=0.781754
INFO:root:Epoch

INFO:root:Epoch[22] Train-accuracy=0.786794
INFO:root:Epoch[22] Time cost=0.323
INFO:root:Epoch[22] Validation-accuracy=0.804167
INFO:root:Epoch[23] Batch [64]	Speed: 50887.94 samples/sec	accuracy=0.808413
INFO:root:Epoch[23] Batch [128]	Speed: 47532.52 samples/sec	accuracy=0.816895
INFO:root:Epoch[23] Batch [192]	Speed: 47463.45 samples/sec	accuracy=0.807861
INFO:root:Epoch[23] Train-accuracy=0.788810
INFO:root:Epoch[23] Time cost=0.302
INFO:root:Epoch[23] Validation-accuracy=0.805208
INFO:root:Update[5401]: Change learning rate to 6.56100e-07
INFO:root:Epoch[24] Batch [64]	Speed: 42440.70 samples/sec	accuracy=0.810577
INFO:root:Epoch[24] Batch [128]	Speed: 42872.93 samples/sec	accuracy=0.818604
INFO:root:Epoch[24] Batch [192]	Speed: 38858.55 samples/sec	accuracy=0.808838
INFO:root:Epoch[24] Train-accuracy=0.790323
INFO:root:Epoch[24] Time cost=0.356
INFO:root:Epoch[24] Validation-accuracy=0.807708
INFO:root:Epoch[25] Batch [64]	Speed: 46193.87 samples/sec	accuracy=0.812981
INFO:root:

INFO:root:Epoch[44] Validation-accuracy=0.831250
INFO:root:Epoch[45] Batch [64]	Speed: 49419.70 samples/sec	accuracy=0.837500
INFO:root:Epoch[45] Batch [128]	Speed: 46868.85 samples/sec	accuracy=0.843750
INFO:root:Epoch[45] Batch [192]	Speed: 48277.99 samples/sec	accuracy=0.836426
INFO:root:Epoch[45] Train-accuracy=0.807460
INFO:root:Epoch[45] Time cost=0.305
INFO:root:Epoch[45] Validation-accuracy=0.831875
INFO:root:Epoch[46] Batch [64]	Speed: 45051.91 samples/sec	accuracy=0.837740
INFO:root:Epoch[46] Batch [128]	Speed: 44124.26 samples/sec	accuracy=0.844482
INFO:root:Epoch[46] Batch [192]	Speed: 44741.81 samples/sec	accuracy=0.836426
INFO:root:Epoch[46] Train-accuracy=0.807460
INFO:root:Epoch[46] Time cost=0.323
INFO:root:Epoch[46] Validation-accuracy=0.832500
INFO:root:Epoch[47] Batch [64]	Speed: 50804.57 samples/sec	accuracy=0.837500
INFO:root:Epoch[47] Batch [128]	Speed: 50070.29 samples/sec	accuracy=0.845215
INFO:root:Epoch[47] Batch [192]	Speed: 49335.68 samples/sec	accuracy=0.8

INFO:root:Epoch[67] Batch [64]	Speed: 44507.89 samples/sec	accuracy=0.845433
INFO:root:Epoch[67] Batch [128]	Speed: 44039.32 samples/sec	accuracy=0.851807
INFO:root:Epoch[67] Batch [192]	Speed: 43180.69 samples/sec	accuracy=0.841553
INFO:root:Epoch[67] Train-accuracy=0.817540
INFO:root:Epoch[67] Time cost=0.332
INFO:root:Epoch[67] Validation-accuracy=0.838125
INFO:root:Epoch[68] Batch [64]	Speed: 37944.68 samples/sec	accuracy=0.845913
INFO:root:Epoch[68] Batch [128]	Speed: 41738.63 samples/sec	accuracy=0.852295
INFO:root:Epoch[68] Batch [192]	Speed: 43367.85 samples/sec	accuracy=0.841309
INFO:root:Epoch[68] Train-accuracy=0.817540
INFO:root:Epoch[68] Time cost=0.355
INFO:root:Epoch[68] Validation-accuracy=0.838125
INFO:root:Epoch[69] Batch [64]	Speed: 36857.10 samples/sec	accuracy=0.846635
INFO:root:Epoch[69] Batch [128]	Speed: 43220.23 samples/sec	accuracy=0.852539
INFO:root:Epoch[69] Batch [192]	Speed: 44418.83 samples/sec	accuracy=0.841064
INFO:root:Epoch[69] Train-accuracy=0.817540

INFO:root:Epoch[89] Batch [128]	Speed: 41526.36 samples/sec	accuracy=0.852051
INFO:root:Epoch[89] Batch [192]	Speed: 44061.00 samples/sec	accuracy=0.848877
INFO:root:Epoch[89] Train-accuracy=0.819556
INFO:root:Epoch[89] Time cost=0.342
INFO:root:Epoch[89] Validation-accuracy=0.842292
INFO:root:Epoch[90] Batch [64]	Speed: 47923.94 samples/sec	accuracy=0.847115
INFO:root:Epoch[90] Batch [128]	Speed: 48532.90 samples/sec	accuracy=0.852295
INFO:root:Epoch[90] Batch [192]	Speed: 47598.24 samples/sec	accuracy=0.848633
INFO:root:Epoch[90] Train-accuracy=0.820060
INFO:root:Epoch[90] Time cost=0.304
INFO:root:Epoch[90] Validation-accuracy=0.842500
INFO:root:Epoch[91] Batch [64]	Speed: 46066.04 samples/sec	accuracy=0.847356
INFO:root:Epoch[91] Batch [128]	Speed: 46550.47 samples/sec	accuracy=0.852295
INFO:root:Epoch[91] Batch [192]	Speed: 43301.28 samples/sec	accuracy=0.848633
INFO:root:Epoch[91] Train-accuracy=0.820565
INFO:root:Epoch[91] Time cost=0.323
INFO:root:Epoch[91] Validation-accuracy=

...

INFO:root:Epoch[220] Train-accuracy=0.832157
INFO:root:Epoch[220] Time cost=0.331
INFO:root:Epoch[220] Validation-accuracy=0.844375
INFO:root:Epoch[221] Batch [64]	Speed: 41740.45 samples/sec	accuracy=0.857212
INFO:root:Epoch[221] Batch [128]	Speed: 41087.88 samples/sec	accuracy=0.859863
INFO:root:Epoch[221] Batch [192]	Speed: 41304.41 samples/sec	accuracy=0.854248
INFO:root:Epoch[221] Train-accuracy=0.832157
INFO:root:Epoch[221] Time cost=0.354
INFO:root:Epoch[221] Validation-accuracy=0.844375
INFO:root:Epoch[222] Batch [64]	Speed: 41517.22 samples/sec	accuracy=0.857212
INFO:root:Epoch[222] Batch [128]	Speed: 35290.27 samples/sec	accuracy=0.860107
INFO:root:Epoch[222] Batch [192]	Speed: 41218.10 samples/sec	accuracy=0.854492
INFO:root:Epoch[222] Train-accuracy=0.832157
INFO:root:Epoch[222] Time cost=0.369
INFO:root:Epoch[222] Validation-accuracy=0.844375
INFO:root:Epoch[223] Batch [64]	Speed: 38217.06 samples/sec	accuracy=0.857212
INFO:root:Epoch[223] Batch [128]	Speed: 35825.05 sampl

INFO:root:Epoch[242] Batch [192]	Speed: 43320.17 samples/sec	accuracy=0.854980
INFO:root:Epoch[242] Train-accuracy=0.831653
INFO:root:Epoch[242] Time cost=0.321
INFO:root:Epoch[242] Validation-accuracy=0.843750
INFO:root:Epoch[243] Batch [64]	Speed: 48027.37 samples/sec	accuracy=0.857212
INFO:root:Epoch[243] Batch [128]	Speed: 44558.69 samples/sec	accuracy=0.860352
INFO:root:Epoch[243] Batch [192]	Speed: 43768.14 samples/sec	accuracy=0.854980
INFO:root:Epoch[243] Train-accuracy=0.831653
INFO:root:Epoch[243] Time cost=0.324
INFO:root:Epoch[243] Validation-accuracy=0.843750
INFO:root:Epoch[244] Batch [64]	Speed: 48328.52 samples/sec	accuracy=0.857212
INFO:root:Epoch[244] Batch [128]	Speed: 45815.92 samples/sec	accuracy=0.860352
INFO:root:Epoch[244] Batch [192]	Speed: 45371.61 samples/sec	accuracy=0.854736
INFO:root:Epoch[244] Train-accuracy=0.831653
INFO:root:Epoch[244] Time cost=0.314
INFO:root:Epoch[244] Validation-accuracy=0.843750
INFO:root:Epoch[245] Batch [64]	Speed: 45758.93 sampl

INFO:root:Epoch[264] Batch [192]	Speed: 43033.59 samples/sec	accuracy=0.855469
INFO:root:Epoch[264] Train-accuracy=0.831653
INFO:root:Epoch[264] Time cost=0.349
INFO:root:Epoch[264] Validation-accuracy=0.843542
INFO:root:Update[59401]: Change learning rate to 2.78128e-08
INFO:root:Epoch[265] Batch [64]	Speed: 40616.46 samples/sec	accuracy=0.856971
INFO:root:Epoch[265] Batch [128]	Speed: 41820.52 samples/sec	accuracy=0.860352
INFO:root:Epoch[265] Batch [192]	Speed: 42875.39 samples/sec	accuracy=0.855469
INFO:root:Epoch[265] Train-accuracy=0.831653
INFO:root:Epoch[265] Time cost=0.346
INFO:root:Epoch[265] Validation-accuracy=0.843542
INFO:root:Epoch[266] Batch [64]	Speed: 40351.63 samples/sec	accuracy=0.856971
INFO:root:Epoch[266] Batch [128]	Speed: 39212.08 samples/sec	accuracy=0.860352
INFO:root:Epoch[266] Batch [192]	Speed: 41960.56 samples/sec	accuracy=0.855469
INFO:root:Epoch[266] Train-accuracy=0.831653
INFO:root:Epoch[266] Time cost=0.354
INFO:root:Epoch[266] Validation-accuracy=0

INFO:root:Epoch[286] Batch [128]	Speed: 39937.39 samples/sec	accuracy=0.860352
INFO:root:Epoch[286] Batch [192]	Speed: 44911.87 samples/sec	accuracy=0.856201
INFO:root:Epoch[286] Train-accuracy=0.831653
INFO:root:Epoch[286] Time cost=0.340
INFO:root:Epoch[286] Validation-accuracy=0.843750
INFO:root:Epoch[287] Batch [64]	Speed: 43596.87 samples/sec	accuracy=0.857212
INFO:root:Epoch[287] Batch [128]	Speed: 39550.69 samples/sec	accuracy=0.860352
INFO:root:Epoch[287] Batch [192]	Speed: 40312.34 samples/sec	accuracy=0.856201
INFO:root:Epoch[287] Train-accuracy=0.831653
INFO:root:Epoch[287] Time cost=0.354
INFO:root:Epoch[287] Validation-accuracy=0.843750
INFO:root:Epoch[288] Batch [64]	Speed: 44166.80 samples/sec	accuracy=0.856971
INFO:root:Epoch[288] Batch [128]	Speed: 44725.96 samples/sec	accuracy=0.860352
INFO:root:Epoch[288] Batch [192]	Speed: 42956.98 samples/sec	accuracy=0.856201
INFO:root:Epoch[288] Train-accuracy=0.831653
INFO:root:Epoch[288] Time cost=0.334
INFO:root:Epoch[288] Val

Accuracy: 84.39583333333334 %


In [16]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'DNN_SVM'
epoch = 300
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "DNN_SVM-0300.params"


In [17]:
modelMetric(mod, 'Finish predict of DNN+SVM...')

INFO:root:Finish predict of DNN+SVM...
INFO:root:Test accuracy = 0.843449


             precision    recall  f1-score   support

        0.0       0.88      0.83      0.85      2645
        1.0       0.80      0.87      0.83      2133

avg / total       0.85      0.84      0.84      4778

[[2184  461]
 [ 287 1846]]


## 2.3 Deeper DNN with dropout + SVM

In [18]:
# Deeper DNN with dropout + SVM
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=1024)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 1024)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
drop1 = mx.sym.Dropout(data=act2, p=0.5)

fc3  = mx.symbol.FullyConnected(data = drop1, name = 'fc3', num_hidden = 512)
act3 = mx.symbol.Activation(data = fc3, name='relu3', act_type="relu")
fc4  = mx.symbol.FullyConnected(data = act3, name = 'fc4', num_hidden = 512)
act4 = mx.symbol.Activation(data = fc4, name='relu4', act_type="relu")

drop2 = mx.sym.Dropout(data=act4, p=0.5)

fc5  = mx.symbol.FullyConnected(data = drop2, name='fc5', num_hidden=2)

ddnn = mx.symbol.SVMOutput(data=fc5, name='svm')

# draw network chart
mx.viz.plot_network(symbol=ddnn, shape=shape)
network_chart=mx.viz.plot_network(symbol=ddnn, shape=shape)
network_chart.format = 'png'
network_chart.render('DDNN + SVM')

'DDNN + SVM.png'

<img src='DDNN + SVM.png'>

In [20]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = ddnn,         # Use the network we just defined
    label_names = ['svm_label'],
)
lr_sch = mx.lr_scheduler.FactorScheduler(step=1800, factor=0.9)

mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size), 
    num_epoch = 300,       # Train for 100 epochs
    optimizer = 'Adam',
    optimizer_params = {
        'learning_rate': 5e-7, # Learning rate
        'beta1': 0.9,       # Beta 1
        'beta2': 0.999,         # Beta 2
        'lr_scheduler': lr_sch #learning rate decrease schedule
    },
)
print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 26610.74 samples/sec	accuracy=0.559615
INFO:root:Epoch[0] Batch [128]	Speed: 26741.22 samples/sec	accuracy=0.567627
INFO:root:Epoch[0] Train-accuracy=0.558036
INFO:root:Epoch[0] Time cost=0.340
INFO:root:Epoch[0] Validation-accuracy=0.553750
INFO:root:Epoch[1] Batch [64]	Speed: 28249.77 samples/sec	accuracy=0.547837
INFO:root:Epoch[1] Batch [128]	Speed: 26692.86 samples/sec	accuracy=0.545166
INFO:root:Epoch[1] Batch [192]	Speed: 29096.72 samples/sec	accuracy=0.553467
INFO:root:Epoch[1] Train-accuracy=0.559980
INFO:root:Epoch[1] Time cost=0.518
INFO:root:Epoch[1] Validation-accuracy=0.553750
INFO:root:Epoch[2] Batch [64]	Speed: 27150.34 samples/sec	accuracy=0.546154
INFO:root:Epoch[2] Batch [128]	Speed: 25583.78 samples/sec	accuracy=0.544678
INFO:root:Epoch[2] Batch [192]	Speed: 28781.97 samples/sec	accuracy=0.552979
INFO:root:Epoch[2] Train-accuracy=0.559980
INFO:root:Epoch[2] Time cost=0.529
INFO:root:Epoch[2] Validation-accuracy=0.553750
INFO:root

INFO:root:Epoch[22] Time cost=0.460
INFO:root:Epoch[22] Validation-accuracy=0.799375
INFO:root:Epoch[23] Batch [64]	Speed: 19753.86 samples/sec	accuracy=0.802885
INFO:root:Epoch[23] Batch [128]	Speed: 27699.06 samples/sec	accuracy=0.812988
INFO:root:Epoch[23] Batch [192]	Speed: 28928.67 samples/sec	accuracy=0.803711
INFO:root:Epoch[23] Train-accuracy=0.780746
INFO:root:Epoch[23] Time cost=0.603
INFO:root:Epoch[23] Validation-accuracy=0.801250
INFO:root:Epoch[24] Batch [64]	Speed: 22585.51 samples/sec	accuracy=0.805529
INFO:root:Update[5401]: Change learning rate to 3.64500e-07
INFO:root:Epoch[24] Batch [128]	Speed: 31074.93 samples/sec	accuracy=0.813721
INFO:root:Epoch[24] Batch [192]	Speed: 29847.93 samples/sec	accuracy=0.805908
INFO:root:Epoch[24] Train-accuracy=0.785282
INFO:root:Epoch[24] Time cost=0.519
INFO:root:Epoch[24] Validation-accuracy=0.803542
INFO:root:Epoch[25] Batch [64]	Speed: 27870.58 samples/sec	accuracy=0.809375
INFO:root:Epoch[25] Batch [128]	Speed: 32234.52 sample

INFO:root:Epoch[45] Batch [64]	Speed: 27502.58 samples/sec	accuracy=0.835577
INFO:root:Epoch[45] Batch [128]	Speed: 30658.90 samples/sec	accuracy=0.840820
INFO:root:Epoch[45] Batch [192]	Speed: 31295.36 samples/sec	accuracy=0.834961
INFO:root:Epoch[45] Train-accuracy=0.805948
INFO:root:Epoch[45] Time cost=0.488
INFO:root:Epoch[45] Validation-accuracy=0.832292
INFO:root:Epoch[46] Batch [64]	Speed: 27433.84 samples/sec	accuracy=0.834135
INFO:root:Epoch[46] Batch [128]	Speed: 24724.36 samples/sec	accuracy=0.839600
INFO:root:Epoch[46] Batch [192]	Speed: 29611.27 samples/sec	accuracy=0.835449
INFO:root:Epoch[46] Train-accuracy=0.804435
INFO:root:Epoch[46] Time cost=0.522
INFO:root:Epoch[46] Validation-accuracy=0.832083
INFO:root:Epoch[47] Batch [64]	Speed: 28706.80 samples/sec	accuracy=0.835096
INFO:root:Epoch[47] Batch [128]	Speed: 31116.69 samples/sec	accuracy=0.840576
INFO:root:Epoch[47] Batch [192]	Speed: 25043.03 samples/sec	accuracy=0.832764
INFO:root:Epoch[47] Train-accuracy=0.807460

INFO:root:Epoch[67] Batch [128]	Speed: 30476.65 samples/sec	accuracy=0.848633
INFO:root:Epoch[67] Batch [192]	Speed: 32724.37 samples/sec	accuracy=0.842529
INFO:root:Epoch[67] Train-accuracy=0.816532
INFO:root:Epoch[67] Time cost=0.495
INFO:root:Epoch[67] Validation-accuracy=0.838958
INFO:root:Epoch[68] Batch [64]	Speed: 30532.38 samples/sec	accuracy=0.843750
INFO:root:Epoch[68] Batch [128]	Speed: 32354.53 samples/sec	accuracy=0.851318
INFO:root:Epoch[68] Batch [192]	Speed: 31958.68 samples/sec	accuracy=0.840332
INFO:root:Epoch[68] Train-accuracy=0.818548
INFO:root:Epoch[68] Time cost=0.457
INFO:root:Epoch[68] Validation-accuracy=0.839375
INFO:root:Epoch[69] Batch [64]	Speed: 30384.37 samples/sec	accuracy=0.844471
INFO:root:Epoch[69] Batch [128]	Speed: 30694.12 samples/sec	accuracy=0.849121
INFO:root:Epoch[69] Batch [192]	Speed: 24403.60 samples/sec	accuracy=0.840820
INFO:root:Epoch[69] Train-accuracy=0.813004
INFO:root:Epoch[69] Time cost=0.541
INFO:root:Epoch[69] Validation-accuracy=

INFO:root:Epoch[89] Batch [192]	Speed: 30300.02 samples/sec	accuracy=0.846924
INFO:root:Epoch[89] Train-accuracy=0.820565
INFO:root:Epoch[89] Time cost=0.483
INFO:root:Epoch[89] Validation-accuracy=0.842292
INFO:root:Epoch[90] Batch [64]	Speed: 28302.26 samples/sec	accuracy=0.848798
INFO:root:Epoch[90] Batch [128]	Speed: 31701.04 samples/sec	accuracy=0.853516
INFO:root:Epoch[90] Batch [192]	Speed: 24446.18 samples/sec	accuracy=0.845215
INFO:root:Epoch[90] Train-accuracy=0.825605
INFO:root:Epoch[90] Time cost=0.548
INFO:root:Epoch[90] Validation-accuracy=0.842708
INFO:root:Epoch[91] Batch [64]	Speed: 20655.14 samples/sec	accuracy=0.849038
INFO:root:Epoch[91] Batch [128]	Speed: 22824.26 samples/sec	accuracy=0.853760
INFO:root:Epoch[91] Batch [192]	Speed: 23369.87 samples/sec	accuracy=0.845215
INFO:root:Epoch[91] Train-accuracy=0.823589
INFO:root:Epoch[91] Time cost=0.658
INFO:root:Epoch[91] Validation-accuracy=0.842917
INFO:root:Epoch[92] Batch [64]	Speed: 21341.45 samples/sec	accuracy=0

...

INFO:root:Epoch[220] Validation-accuracy=0.846875
INFO:root:Epoch[221] Batch [64]	Speed: 29166.37 samples/sec	accuracy=0.858894
INFO:root:Epoch[221] Batch [128]	Speed: 30802.87 samples/sec	accuracy=0.861816
INFO:root:Epoch[221] Batch [192]	Speed: 30691.27 samples/sec	accuracy=0.852295
INFO:root:Epoch[221] Train-accuracy=0.830141
INFO:root:Epoch[221] Time cost=0.486
INFO:root:Epoch[221] Validation-accuracy=0.846875
INFO:root:Epoch[222] Batch [64]	Speed: 26962.98 samples/sec	accuracy=0.856250
INFO:root:Epoch[222] Batch [128]	Speed: 24142.19 samples/sec	accuracy=0.862305
INFO:root:Epoch[222] Batch [192]	Speed: 23963.21 samples/sec	accuracy=0.854248
INFO:root:Epoch[222] Train-accuracy=0.831653
INFO:root:Epoch[222] Time cost=0.598
INFO:root:Epoch[222] Validation-accuracy=0.846875
INFO:root:Epoch[223] Batch [64]	Speed: 28096.89 samples/sec	accuracy=0.858894
INFO:root:Epoch[223] Batch [128]	Speed: 28429.98 samples/sec	accuracy=0.859863
INFO:root:Epoch[223] Batch [192]	Speed: 30574.11 samples/

INFO:root:Epoch[242] Train-accuracy=0.828125
INFO:root:Epoch[242] Time cost=0.492
INFO:root:Epoch[242] Validation-accuracy=0.846250
INFO:root:Epoch[243] Batch [64]	Speed: 28760.28 samples/sec	accuracy=0.859135
INFO:root:Epoch[243] Batch [128]	Speed: 32241.05 samples/sec	accuracy=0.862061
INFO:root:Epoch[243] Batch [192]	Speed: 31045.23 samples/sec	accuracy=0.856201
INFO:root:Epoch[243] Train-accuracy=0.833165
INFO:root:Epoch[243] Time cost=0.470
INFO:root:Epoch[243] Validation-accuracy=0.846250
INFO:root:Epoch[244] Batch [64]	Speed: 29700.19 samples/sec	accuracy=0.859135
INFO:root:Epoch[244] Batch [128]	Speed: 31180.51 samples/sec	accuracy=0.861572
INFO:root:Epoch[244] Batch [192]	Speed: 31535.95 samples/sec	accuracy=0.853271
INFO:root:Epoch[244] Train-accuracy=0.827117
INFO:root:Epoch[244] Time cost=0.470
INFO:root:Epoch[244] Validation-accuracy=0.846250
INFO:root:Epoch[245] Batch [64]	Speed: 28595.99 samples/sec	accuracy=0.856010
INFO:root:Epoch[245] Batch [128]	Speed: 31817.93 sampl

INFO:root:Epoch[264] Train-accuracy=0.831653
INFO:root:Epoch[264] Time cost=0.516
INFO:root:Epoch[264] Validation-accuracy=0.846250
INFO:root:Epoch[265] Batch [64]	Speed: 30310.39 samples/sec	accuracy=0.861058
INFO:root:Update[59401]: Change learning rate to 1.54516e-08
INFO:root:Epoch[265] Batch [128]	Speed: 28673.07 samples/sec	accuracy=0.864258
INFO:root:Epoch[265] Batch [192]	Speed: 25639.91 samples/sec	accuracy=0.857422
INFO:root:Epoch[265] Train-accuracy=0.831149
INFO:root:Epoch[265] Time cost=0.537
INFO:root:Epoch[265] Validation-accuracy=0.846250
INFO:root:Epoch[266] Batch [64]	Speed: 30475.94 samples/sec	accuracy=0.858654
INFO:root:Epoch[266] Batch [128]	Speed: 30576.24 samples/sec	accuracy=0.861572
INFO:root:Epoch[266] Batch [192]	Speed: 29770.60 samples/sec	accuracy=0.858154
INFO:root:Epoch[266] Train-accuracy=0.828629
INFO:root:Epoch[266] Time cost=0.483
INFO:root:Epoch[266] Validation-accuracy=0.846250
INFO:root:Epoch[267] Batch [64]	Speed: 23241.76 samples/sec	accuracy=0.

INFO:root:Epoch[286] Batch [192]	Speed: 26481.25 samples/sec	accuracy=0.851562
INFO:root:Epoch[286] Train-accuracy=0.829637
INFO:root:Epoch[286] Time cost=0.524
INFO:root:Epoch[286] Validation-accuracy=0.846458
INFO:root:Epoch[287] Batch [64]	Speed: 22936.80 samples/sec	accuracy=0.859375
INFO:root:Epoch[287] Batch [128]	Speed: 21246.49 samples/sec	accuracy=0.859375
INFO:root:Epoch[287] Batch [192]	Speed: 28788.62 samples/sec	accuracy=0.855713
INFO:root:Epoch[287] Train-accuracy=0.834677
INFO:root:Epoch[287] Time cost=0.588
INFO:root:Epoch[287] Validation-accuracy=0.846667
INFO:root:Epoch[288] Batch [64]	Speed: 25526.35 samples/sec	accuracy=0.857452
INFO:root:Epoch[288] Batch [128]	Speed: 22570.55 samples/sec	accuracy=0.859375
INFO:root:Epoch[288] Batch [192]	Speed: 30630.59 samples/sec	accuracy=0.858398
INFO:root:Epoch[288] Train-accuracy=0.830645
INFO:root:Epoch[288] Time cost=0.553
INFO:root:Epoch[288] Validation-accuracy=0.846667
INFO:root:Epoch[289] Batch [64]	Speed: 28125.54 sampl

Accuracy: 84.625 %


In [21]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'DDNN_SVM'
epoch = 300
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "DDNN_SVM-0300.params"


In [22]:
modelMetric(mod, 'Finish predict of DDNN + SVM...')

INFO:root:Finish predict of DDNN + SVM...
INFO:root:Test accuracy = 0.845751


             precision    recall  f1-score   support

        0.0       0.89      0.82      0.85      2645
        1.0       0.80      0.88      0.84      2133

avg / total       0.85      0.85      0.85      4778

[[2169  476]
 [ 261 1872]]


## 2.4 Inception Batach Normalization

In [23]:
# Reference: www.mxnet.io
# Basic Conv + BN + ReLU factory
def ConvFactory(data, num_filter, kernel, stride=(1,1), pad=(0, 0), act_type="relu"):
    conv = mx.symbol.Convolution(data=data, workspace=256,
                                 num_filter=num_filter, kernel=kernel, stride=stride, pad=pad)
    bn = mx.symbol.BatchNorm(data=conv)
    act = mx.symbol.Activation(data = bn, act_type=act_type)
    return act

# A Simple Downsampling Factory
def DownsampleFactory(data, ch_3x3):
    conv = ConvFactory(data=data, kernel=(3, 3), stride=(2, 2), num_filter=ch_3x3, pad=(1, 1))
    pool = mx.symbol.Pooling(data=data, kernel=(3, 3), stride=(2, 2),pad=(1,1), pool_type='max')
    concat = mx.symbol.Concat(*[conv, pool])
    return concat

# A Simple module
def SimpleFactory(data, ch_1x1, ch_3x3):
    conv1x1 = ConvFactory(data=data, kernel=(1, 1), pad=(0, 0), num_filter=ch_1x1)
    conv3x3 = ConvFactory(data=data, kernel=(3, 3), pad=(1, 1), num_filter=ch_3x3)
    concat = mx.symbol.Concat(*[conv1x1, conv3x3])
    return concat

def InceptionFactory(data, num_1x1, num_3x3red, num_3x3, num_d5x5red, num_d5x5, pool, proj, name):
    c1x1 = ConvFactoryB(data=data, num_filter=num_1x1, kernel=(1, 1), name=('%s_1x1' % name))
    c3x3r = ConvFactoryB(data=data, num_filter=num_3x3red, kernel=(1, 1), name=('%s_3x3' % name), suffix='_reduce')
    c3x3 = ConvFactoryB(data=c3x3r, num_filter=num_3x3, kernel=(3, 3), pad=(1, 1), name=('%s_3x3' % name))
    cd5x5r = ConvFactoryB(data=data, num_filter=num_d5x5red, kernel=(1, 1), name=('%s_double_3x3' % name), suffix='_reduce')
    cd5x5 = ConvFactoryB(data=cd5x5r, num_filter=num_d5x5, kernel=(5, 5), pad=(2, 2), name=('%s_double_3x3_1' % name))
    pooling = mx.symbol.Pooling(data=data, kernel=(3, 3), stride=(1, 1), pad=(1, 1), pool_type=pool, name=('%s_pool_%s_pool' % (pool, name)))
    cproj = ConvFactoryB(data=pooling, num_filter=proj, kernel=(1, 1), name=('%s_proj' %  name))
    concat = mx.symbol.Concat(*[c1x1, c3x3, cd5x5, cproj], name='ch_concat_%s_chconcat' % name)
    return concat

In [26]:
# Inception Batch Normalization Network
#https://arxiv.org/pdf/1502.03167.pdf
data = mx.sym.Variable('data')

conv1 = ConvFactory(data=data, kernel=(3,3), pad=(1,1), num_filter=96, act_type="relu")
in3a = SimpleFactory(conv1, 32, 32)
in3b = SimpleFactory(in3a, 32, 48)
in3c = DownsampleFactory(in3b, 80)
in4a = SimpleFactory(in3c, 112, 48)
in4b = SimpleFactory(in4a, 96, 64)
in4c = SimpleFactory(in4b, 80, 80)
in4d = SimpleFactory(in4c, 48, 96)
in4e = DownsampleFactory(in4d, 96)
in5a = SimpleFactory(in4e, 176, 160)
in5b = SimpleFactory(in5a, 176, 160)
pool = mx.symbol.Pooling(data=in5b, pool_type="avg", kernel=(3,3), name="global_pool")
flatten = mx.symbol.Flatten(data=pool, name="flatten1")
fc0  = mx.symbol.FullyConnected(data = flatten, name = 'fc0', num_hidden = 10)
act0 = mx.symbol.Activation(data = fc0, name='relu0', act_type="relu")
fc = mx.symbol.FullyConnected(data=act0, num_hidden=2, name="fc1")
ibn = mx.symbol.SVMOutput(data=fc, name='svm')

# draw network chart
mx.viz.plot_network(symbol=ibn, shape=shape)
network_chart=mx.viz.plot_network(symbol=ibn, shape=shape)
network_chart.format = 'png'
network_chart.render('Inception Batch Normalization')

'Inception Batch Normalization.png'

<img src = 'Inception Batch Normalization.png'>

In [33]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = ibn,         # Use the network we just defined
    label_names = ['svm_label'],
)

mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size),  
    num_epoch = 50,       # Train for 50 epochs
    optimizer_params = {
        'learning_rate': 1e-4,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.00001,         # Weight decay for regularization
    },
)

print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 2986.45 samples/sec	accuracy=0.544712
INFO:root:Epoch[0] Train-accuracy=0.556818
INFO:root:Epoch[0] Time cost=2.571
INFO:root:Epoch[0] Validation-accuracy=0.553750
INFO:root:Epoch[1] Batch [64]	Speed: 3136.23 samples/sec	accuracy=0.546154
INFO:root:Epoch[1] Batch [128]	Speed: 2960.71 samples/sec	accuracy=0.544678
INFO:root:Epoch[1] Batch [192]	Speed: 3125.75 samples/sec	accuracy=0.552979
INFO:root:Epoch[1] Train-accuracy=0.559980
INFO:root:Epoch[1] Time cost=4.646
INFO:root:Epoch[1] Validation-accuracy=0.553750
INFO:root:Epoch[2] Batch [64]	Speed: 3139.86 samples/sec	accuracy=0.546154
INFO:root:Epoch[2] Batch [128]	Speed: 3113.90 samples/sec	accuracy=0.656006
INFO:root:Epoch[2] Batch [192]	Speed: 2859.69 samples/sec	accuracy=0.773438
INFO:root:Epoch[2] Train-accuracy=0.778730
INFO:root:Epoch[2] Time cost=4.721
INFO:root:Epoch[2] Validation-accuracy=0.809792
INFO:root:Epoch[3] Batch [64]	Speed: 3141.02 samples/sec	accuracy=0.818269
INFO:root:Epoch[3]

INFO:root:Epoch[23] Batch [192]	Speed: 3097.17 samples/sec	accuracy=0.994385
INFO:root:Epoch[23] Train-accuracy=0.990423
INFO:root:Epoch[23] Time cost=4.579
INFO:root:Epoch[23] Validation-accuracy=0.834167
INFO:root:Epoch[24] Batch [64]	Speed: 3146.26 samples/sec	accuracy=0.991587
INFO:root:Epoch[24] Batch [128]	Speed: 3126.29 samples/sec	accuracy=0.990967
INFO:root:Epoch[24] Batch [192]	Speed: 3173.89 samples/sec	accuracy=0.993408
INFO:root:Epoch[24] Train-accuracy=0.992440
INFO:root:Epoch[24] Time cost=4.544
INFO:root:Epoch[24] Validation-accuracy=0.830000
INFO:root:Epoch[25] Batch [64]	Speed: 3199.65 samples/sec	accuracy=0.990865
INFO:root:Epoch[25] Batch [128]	Speed: 3181.77 samples/sec	accuracy=0.992188
INFO:root:Epoch[25] Batch [192]	Speed: 3141.73 samples/sec	accuracy=0.994385
INFO:root:Epoch[25] Train-accuracy=0.991935
INFO:root:Epoch[25] Time cost=4.537
INFO:root:Epoch[25] Validation-accuracy=0.833750
INFO:root:Epoch[26] Batch [64]	Speed: 3087.44 samples/sec	accuracy=0.991106


INFO:root:Epoch[46] Batch [192]	Speed: 3160.44 samples/sec	accuracy=0.999756
INFO:root:Epoch[46] Train-accuracy=0.999496
INFO:root:Epoch[46] Time cost=4.572
INFO:root:Epoch[46] Validation-accuracy=0.812917
INFO:root:Epoch[47] Batch [64]	Speed: 3104.67 samples/sec	accuracy=0.999519
INFO:root:Epoch[47] Batch [128]	Speed: 3100.52 samples/sec	accuracy=0.998535
INFO:root:Epoch[47] Batch [192]	Speed: 3076.54 samples/sec	accuracy=0.998535
INFO:root:Epoch[47] Train-accuracy=0.998488
INFO:root:Epoch[47] Time cost=4.625
INFO:root:Epoch[47] Validation-accuracy=0.827500
INFO:root:Epoch[48] Batch [64]	Speed: 3145.35 samples/sec	accuracy=0.999519
INFO:root:Epoch[48] Batch [128]	Speed: 3169.35 samples/sec	accuracy=1.000000
INFO:root:Epoch[48] Batch [192]	Speed: 3171.42 samples/sec	accuracy=0.999756
INFO:root:Epoch[48] Train-accuracy=1.000000
INFO:root:Epoch[48] Time cost=4.523
INFO:root:Epoch[48] Validation-accuracy=0.820208
INFO:root:Epoch[49] Batch [64]	Speed: 3168.74 samples/sec	accuracy=1.000000


Accuracy: 83.29166666666666 %


In [34]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'Inception_BN'
epoch = 50
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "Inception_BN-0050.params"


In [35]:
modelMetric(mod, 'Finish predict of Inception BN..')

INFO:root:Finish predict of Inception BN..
INFO:root:Test accuracy = 0.832566


             precision    recall  f1-score   support

        0.0       0.86      0.83      0.85      2645
        1.0       0.80      0.83      0.82      2133

avg / total       0.83      0.83      0.83      4778

[[2206  439]
 [ 361 1772]]
